# Remind if there is impending stormy weather

In [1]:
# import library
import datetime
import commands
import pandas as pd
import dateutil.parser

### Read data

In [2]:
#project_home = os.environ['project_home'] # <- actual run
project_home = '/home/scube_backend/airflow_analysis_pipe'
datadir = project_home + '/data/'
cmd = 'ls {datadir}/sg_weather_data.csv'.format(datadir=datadir)
filename = commands.getoutput(cmd).splitlines()[-1]
print 'filename:', filename

filename: /home/scube_backend/airflow_analysis_pipe/data//sg_weather_data.csv


In [3]:
df = pd.read_csv(filename, sep='|')
df

,timestamp,high,low,description
0,2018-04-15T11:34:00,34.0,24.0,Thundery showers over many areas in the aftern...


### Run algorithm

In [4]:
# extract today
_today = datetime.datetime.now().isoformat().split('T')[0]
dets = [_.split('T')[0] == _today for _ in df['timestamp']]
df_today = df[dets]
df_today

,timestamp,high,low,description
0,2018-04-15T11:34:00,34.0,24.0,Thundery showers over many areas in the aftern...


In [5]:
# find latest record
rec = df_today.sort_values(by='timestamp').iloc[-1].to_dict()
rec

{'description': 'Thundery showers over many areas in the afternoon.',
 'high': 34.0,
 'low': 24.0,
 'timestamp': '2018-04-15T11:34:00'}

In [6]:
# decide reminder
equipments = []

#  - umbrella
desc_text = rec['description'].upper()
keywords = ['RAIN', 'SHOWER', 'STORM']
det = any([_ in desc_text for _ in keywords])
if det == True:
    equipments.append('umbrella')
# end if

# - coat
if rec['low'] < 25:
    equipments.append('coat')
# end if

# - drink
if rec['high'] > 35:
    equipments.append('icy drink')
# end if

In [7]:
print 'equipment:', equipments

equipment: ['umbrella', 'coat']


### Export to file

In [8]:
timestamp = datetime.datetime.now().isoformat()
content = ','.join(equipments)
line = '\t'.join([timestamp, content])

In [9]:
# append to a "pipe" file
output_dir  = project_home + '/data/'
output_file = '{output_dir}/equipment_alert.csv'.format(output_dir=output_dir)

with open(output_file, 'a') as fout:
    fout.writelines(line+'\n')
# end with